In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import time
import random

In [3]:
!ls data

k49-test-imgs.npz    k49-train-imgs.npz
k49-test-labels.npz  k49-train-labels.npz


In [4]:
"""
import zipfile

hiragana73_path = '../dataset/hira_gen/hiragana73.zip'
with zipfile.ZipFile(hiragana73_path) as existing_zip:
    existing_zip.extractall(hiragana73_path[:-4])
hiragana73s = os.listdir(hiragana73_path[:-4])
hiragana73_dirs = [f for f in hiragana73s if os.path.isdir(os.path.join(hiragana73_path[:-4], f))]
print(hiragana73_dirs) 

hiragana_imgs_path = '/content/drive/MyDrive/hira_gen/hiragana_imgs.zip'
with zipfile.ZipFile(hiragana_imgs_path) as existing_zip:
    existing_zip.extractall(hiragana_imgs_path[:-4])
hiragana_imgs = os.listdir(hiragana_imgs_path[:-4])
hiragana_img_dirs = [f for f in hiragana_imgs if os.path.isdir(os.path.join(hiragana_imgs_path[:-4], f))]
print(hiragana_img_dirs) 

# get label
# あーん
all_label_classes = set([os.path.basename(d) for d in hiragana73_dirs] + [os.path.basename(d) for d in hiragana_img_dirs])
# 0-
all_labels = [i for i in range(len(all_label_classes))]
print(all_label_classes)
print(len(all_label_classes))
print(all_labels)
"""

"\nimport zipfile\n\nhiragana73_path = '../dataset/hira_gen/hiragana73.zip'\nwith zipfile.ZipFile(hiragana73_path) as existing_zip:\n    existing_zip.extractall(hiragana73_path[:-4])\nhiragana73s = os.listdir(hiragana73_path[:-4])\nhiragana73_dirs = [f for f in hiragana73s if os.path.isdir(os.path.join(hiragana73_path[:-4], f))]\nprint(hiragana73_dirs) \n\nhiragana_imgs_path = '/content/drive/MyDrive/hira_gen/hiragana_imgs.zip'\nwith zipfile.ZipFile(hiragana_imgs_path) as existing_zip:\n    existing_zip.extractall(hiragana_imgs_path[:-4])\nhiragana_imgs = os.listdir(hiragana_imgs_path[:-4])\nhiragana_img_dirs = [f for f in hiragana_imgs if os.path.isdir(os.path.join(hiragana_imgs_path[:-4], f))]\nprint(hiragana_img_dirs) \n\n# get label\n# あーん\nall_label_classes = set([os.path.basename(d) for d in hiragana73_dirs] + [os.path.basename(d) for d in hiragana_img_dirs])\n# 0-\nall_labels = [i for i in range(len(all_label_classes))]\nprint(all_label_classes)\nprint(len(all_label_classes))\np

In [5]:
# create hiragana73 map dict
hiragana73_map = {}
with open('./hiragana_mojigazo_map.txt') as hm:
    lines = hm.readlines()
    for l in lines:
        map_line = l.strip().split(',')
        hiragana73_map[map_line[1]] = map_line[0]
print(hiragana73_map)
print(len(hiragana73_map))

{'U3042': 'あ', 'U3044': 'い', 'U3046': 'う', 'U3048': 'え', 'U304A': 'お', 'U304B': 'か', 'U304C': 'が', 'U304D': 'き', 'U304E': 'ぎ', 'U304F': 'く', 'U3050': 'ぐ', 'U3051': 'け', 'U3052': 'げ', 'U3053': 'こ', 'U3054': 'ご', 'U3055': 'さ', 'U3056': 'ざ', 'U3057': 'し', 'U3058': 'じ', 'U3059': 'す', 'U305A': 'ず', 'U305B': 'せ', 'U305C': 'ぜ', 'U305D': 'そ', 'U305E': 'ぞ', 'U305F': 'た', 'U3060': 'だ', 'U3061': 'ち', 'U3062': 'ぢ', 'U3064': 'つ', 'U3065': 'づ', 'U3066': 'て', 'U3067': 'で', 'U3068': 'と', 'U3069': 'ど', 'U306A': 'な', 'U306B': 'に', 'U306C': 'ぬ', 'U306D': 'ね', 'U306E': 'の', 'U306F': 'は', 'U3070': 'ば', 'U3071': 'ぱ', 'U3072': 'ひ', 'U3073': 'び', 'U3074': 'ぴ', 'U3075': 'ふ', 'U3076': 'ぶ', 'U3077': 'ぷ', 'U3078': 'へ', 'U3079': 'べ', 'U307A': 'ぺ', 'U307B': 'ほ', 'U307C': 'ぼ', 'U307D': 'ぽ', 'U307E': 'ま', 'U307F': 'み', 'U3080': 'む', 'U3081': 'め', 'U3082': 'も', 'U3084': 'や', 'U3086': 'ゆ', 'U3088': 'よ', 'U3089': 'ら', 'U308A': 'り', 'U308B': 'る', 'U308C': 'れ', 'U308D': 'ろ', 'U308F': 'わ', 'U3090': 'ゐ', 'U3091': 'ゑ', 'U309

In [6]:
'げ'=='げ'

True

In [7]:
basedir = '../dataset/hiragana73'
for fn in os.listdir(basedir):
    # print(fn)
    if not os.path.isdir(os.path.join(basedir, fn)):
        continue # Not a directory
    if fn in hiragana73_map.keys():
        newfn  = hiragana73_map[fn]
        # print(newfn)
        os.rename(os.path.join(basedir, fn), os.path.join(basedir, newfn))

In [8]:
hiragana73_path = '../dataset/hiragana73'

hiragana73s = os.listdir(hiragana73_path)
hiragana73_dirs = []
for f in hiragana73s:
    # print(f)
    if os.path.isdir(os.path.join(hiragana73_path, f)):
        hiragana73_dirs.append(f)
print(hiragana73_dirs)
print(len(hiragana73_dirs))

hiragana_imgs_path = '../dataset/hiragana_imgs'

hiragana_imgs = os.listdir(hiragana_imgs_path)
hiragana_img_dirs = [f for f in hiragana_imgs if os.path.isdir(os.path.join(hiragana_imgs_path, f))]
print(hiragana_img_dirs) 
print(len(hiragana_img_dirs))

# get label
# あーん
all_label_classes = list(set([os.path.basename(d) for d in hiragana73_dirs] + [os.path.basename(d) for d in hiragana_img_dirs]))
all_label_classes.sort()
# 0-
all_labels = [i for i in range(len(all_label_classes))]
num_label = len(all_label_classes)
print(all_label_classes)
print(num_label)
print(all_labels)

['ぱ', 'え', 'た', 'つ', 'ぷ', 'げ', 'そ', 'ろ', 'だ', 'ぺ', 'り', 'ご', 'ぎ', 'ず', 'よ', 'ね', 'ば', 'ひ', 'ゑ', 'ま', 'や', 'め', 'ん', 'を', 'は', 'む', 'ほ', 'ぶ', 'ふ', 'び', 'と', 'の', 'う', 'て', 'ざ', 'が', 'へ', 'く', 'ぼ', 'ゐ', 'も', 'に', 'わ', 'べ', 'ぬ', 'じ', 'れ', 'ら', 'ぢ', 'す', 'づ', 'ぞ', 'ち', 'け', 'お', 'な', 'い', 'で', 'あ', 'ど', 'き', 'ぜ', 'ぐ', 'ぽ', 'さ', 'こ', 'か', 'せ', 'み', 'る', 'し', 'ゆ', 'ぴ']
73
['ゅ', 'ぱ', 'え', 'た', 'つ', 'ぷ', 'げ', 'そ', 'ろ', 'ゃ', 'だ', 'ぺ', 'り', 'ご', 'ぎ', 'ず', 'よ', 'ね', 'ば', 'ひ', 'ぃ', 'ゑ', 'ま', 'ぁ', 'や', 'め', 'ん', 'を', 'は', 'む', 'ほ', 'ぶ', 'ふ', 'ぉ', 'び', 'と', 'の', 'っ', 'う', 'て', 'ざ', 'が', 'へ', 'ぅ', 'く', 'ぼ', 'ゐ', 'も', 'に', 'わ', 'ゝ', 'べ', 'ぬ', 'ぇ', 'じ', 'れ', 'ら', 'ぢ', 'ょ', 'す', 'づ', 'ぞ', 'ち', 'け', 'お', 'な', 'い', 'で', 'あ', 'ど', 'き', 'ぜ', 'ゎ', 'ぐ', 'ぽ', 'さ', 'こ', 'か', 'せ', 'み', 'ゞ', 'る', 'し', 'ゆ', 'ぴ']
85
['ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', '

In [9]:
import glob

img_dicts = {}
label_dicts = {}
# key: あーん
# value: [all paths]
for cls, label in zip(all_label_classes, all_labels):
  img_dicts[cls] = []
  label_dicts[cls] = str(label)
  # hiragana73_path
  hiragana73_path_target_cls_dir = os.path.join(hiragana73_path, cls)
  if os.path.exists(hiragana73_path_target_cls_dir):
    hiragana73_cls_imgs = glob.glob(hiragana73_path_target_cls_dir + '/*')
  else:
    hiragana73_cls_imgs = []
  print(len(hiragana73_cls_imgs))

  # hiragana_imgs_path
  hiragana_imgs_path_target_cls_dir = os.path.join(hiragana_imgs_path, cls)
  if os.path.exists(hiragana_imgs_path_target_cls_dir):
    hiragana_imgs_path_cls_imgs = glob.glob(hiragana_imgs_path_target_cls_dir + '/*')
  else:
    hiragana_imgs_path_cls_imgs = []
  print(len(hiragana_imgs_path_cls_imgs))
  
  cls_imgs = hiragana73_cls_imgs + hiragana_imgs_path_cls_imgs
  print(len(cls_imgs))

  for cls_img_path in cls_imgs:
    img_dicts[cls].append(cls_img_path)
  print(cls)
  print(len(img_dicts[cls]))

print(label_dicts)

0
380
380
ぁ
380
1208
386
1594
あ
1594
0
380
380
ぃ
380
1122
386
1508
い
1508
0
380
380
ぅ
380
1148
386
1534
う
1534
0
380
380
ぇ
380
1077
386
1463
え
1463
0
380
380
ぉ
380
1283
386
1669
お
1669
1259
386
1645
か
1645
1200
384
1584
が
1584
1121
386
1507
き
1507
1091
384
1475
ぎ
1475
1266
386
1652
く
1652
1043
384
1427
ぐ
1427
1155
386
1541
け
1541
1058
384
1442
げ
1442
1115
391
1506
こ
1506
1078
384
1462
ご
1462
1261
386
1647
さ
1647
1070
379
1449
ざ
1449
1285
386
1671
し
1671
1099
379
1478
じ
1478
1282
386
1668
す
1668
1046
379
1425
ず
1425
1165
386
1551
せ
1551
1115
379
1494
ぜ
1494
1285
386
1671
そ
1671
1066
379
1445
ぞ
1445
1285
386
1671
た
1671
1116
379
1495
だ
1495
1052
386
1438
ち
1438
1132
379
1511
ぢ
1511
0
381
381
っ
381
1142
386
1528
つ
1528
1080
379
1459
づ
1459
1213
386
1599
て
1599
1178
379
1557
で
1557
1184
386
1570
と
1570
1134
379
1513
ど
1513
1233
386
1619
な
1619
1260
386
1646
に
1646
1093
386
1479
ぬ
1479
1126
386
1512
ね
1512
1160
386
1546
の
1546
1247
386
1633
は
1633
1105
379
1484
ば
1484
262
379
641
ぱ
641
1074

In [10]:
# create train paths and label
train_data = []

for hira, fps in img_dicts.items():
    label =int(label_dicts[hira])
    for fp in fps:
        train_data.append([fp, label])
train_data = np.array(train_data)
print(train_data.shape)
print(train_data)

(112570, 2)
[['../dataset/hiragana_imgs/ぁ/AP.jpg' '0']
 ['../dataset/hiragana_imgs/ぁ/ZinHenaKokuryu-RDF.jpg' '0']
 ['../dataset/hiragana_imgs/ぁ/851H-kktt_004.jpg' '0']
 ...
 ['../dataset/hiragana_imgs/ゞ/GenShinGothic-P-ExtraLight.jpg' '84']
 ['../dataset/hiragana_imgs/ゞ/GenEiMGothic2-Medium.jpg' '84']
 ['../dataset/hiragana_imgs/ゞ/GenEiGothicM-Bold.jpg' '84']]


In [11]:
# TODO: gra background to white and gray char to black
# if create all black, then pass

In [12]:
# create nenchan img
# get 'ne, nn, ti, lya'
ne_l = [d[0] for d in train_data if d[1] == label_dicts['ね']]
nn_l = [d[0] for d in train_data if d[1] == label_dicts['ん']]
ti_l = [d[0] for d in train_data if d[1] == label_dicts['ち']]
lya_l = [d[0] for d in train_data if d[1] == label_dicts['ゃ']]

In [13]:
def img_preprcess(fp):
    img = cv2.imread(fp)
    img = cv2.resize(img, (28, 28))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = img_gray.sum() / (img_gray.shape[0] * img_gray.shape[1])
    img_wb = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
    
    return img_wb

In [14]:
import random
import time

train_img = []
data_num = 200000
for i in range(data_num):
    ne_c = img_preprcess(random.choice(ne_l))
    nn_c1 = img_preprcess(random.choice(nn_l))
    ti_c = img_preprcess(random.choice(ti_l))
    lya_c = img_preprcess(random.choice(lya_l))
    nn_c2 = img_preprcess(random.choice(nn_l))
    con_img = cv2.hconcat([ne_c, nn_c1, ti_c, lya_c, nn_c2])
    if (ne_c.sum() == 0) or (nn_c1.sum() == 0) or (ti_c.sum() == 0) or (lya_c.sum() == 0) or (nn_c2.sum() == 0):
        print("has black img")
        cv2.imwrite('sample_black.jpg', con_img)
        continue
    train_img.append(con_img)
    # time.sleep(0.5)
    # cv2.imwrite('sample.jpg', con_img)
cv2.imwrite('sample.jpg', con_img)
train_img = np.array(train_img)
print(train_img.shape)

# background labelもあった方がいいかも？？？
train_label = np.array([0] * len(train_img)).astype(np.uint8)
print(train_label)

(200000, 28, 140)
[0 0 0 ... 0 0 0]


In [15]:
"""
train_img = []
for fp in train_data[0]:
    img = cv2.imread(fp)
    img = cv2.resize(img, (28, 28))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    train_img.append(img)
train_img = np.array(train_img)
print(train_img.shape)
train_label = train_data[1].astype(np.uint8)
print(train_label)"""

'\ntrain_img = []\nfor fp in train_data[0]:\n    img = cv2.imread(fp)\n    img = cv2.resize(img, (28, 28))\n    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n    train_img.append(img)\ntrain_img = np.array(train_img)\nprint(train_img.shape)\ntrain_label = train_data[1].astype(np.uint8)\nprint(train_label)'

In [16]:
"""np.random.shuffle(train_data)
train_data =  train_data.T
print(train_data)"""

'np.random.shuffle(train_data)\ntrain_data =  train_data.T\nprint(train_data)'

In [17]:
train_img.shape

(200000, 28, 140)

In [18]:
train_label

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

# start GAN

In [43]:
img_w, img_h = train_img[0].shape
print(img_w)
print(img_h)

28
140


In [44]:
class Transform(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        sample = np.array(sample, dtype = np.float32)
        sample = torch.tensor(sample)
        return (sample/127.5)-1

transform = Transform()

In [45]:
from tqdm import tqdm

class dataset_full(torch.utils.data.Dataset):

    def __init__(self, img, label, transform=None):
        self.transform = transform
        self.data_num = len(img)
        self.data = []
        self.label = []
        for i in tqdm(range(self.data_num)):
            self.data.append([img[i]])
            self.label.append(label[i])
        self.data_num = len(self.data)

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = np.identity(num_label)[self.label[idx]]
        out_label = np.array(out_label, dtype = np.float32)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label

In [46]:
train_data = dataset_full(train_img, train_label, transform=transform)

100%|██████████| 200000/200000 [00:00<00:00, 683053.00it/s]


In [47]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers=2)

In [48]:
class Generator(nn.Module):
    def __init__(self, z_dim, num_class):
        super(Generator, self).__init__()

        self.fc1 = nn.Linear(z_dim, 300)
        self.bn1 = nn.BatchNorm1d(300)
        self.LReLU1 = nn.LeakyReLU(0.2)

        self.fc2 = nn.Linear(num_class, 1500)
        self.bn2 = nn.BatchNorm1d(1500)
        self.LReLU2 = nn.LeakyReLU(0.2)

        self.fc3 = nn.Linear(1800, 128 * int((img_w / 4)) * int((img_h / 4)))
        self.bn3 = nn.BatchNorm1d(128 * int((img_w / 4)) * int((img_h / 4)))
        self.bo1 = nn.Dropout(p=0.5)
        self.LReLU3 = nn.LeakyReLU(0.2)

        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), #チャネル数を128⇒64に変える。
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1), #チャネル数を64⇒1に変更
            nn.Tanh(),
        )

        self.init_weights()

    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.ConvTranspose2d):
                module.weight.data.normal_(0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.Linear):
                module.weight.data.normal_(0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm1d):
                module.weight.data.normal_(1.0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.normal_(1.0, 0.02)
                module.bias.data.zero_()

    def forward(self, noise, labels):
        y_1 = self.fc1(noise)
        y_1 = self.bn1(y_1)
        y_1 = self.LReLU1(y_1)

        y_2 = self.fc2(labels)
        y_2 = self.bn2(y_2)
        y_2 = self.LReLU2(y_2)

        x = torch.cat([y_1, y_2], 1)
        x = self.fc3(x)
        x = self.bo1(x)
        x = self.LReLU3(x)
        # print(x.shape)
        x = x.view(-1, 128, int((img_w / 4)), int((img_h / 4)))
        # print(x.shape)
        x = self.deconv(x)
        return x


In [49]:
class Discriminator(nn.Module):
    def __init__(self, num_class):
        super(Discriminator, self).__init__()
        self.num_class = num_class

        self.conv = nn.Sequential(
            nn.Conv2d(num_class + 1, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(128),
        )

        self.fc = nn.Sequential(
            nn.Linear(128 * int((img_w / 4)) * int((img_h / 4)), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1),
            nn.Sigmoid(),
        )

        self.init_weights()

    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                module.weight.data.normal_(0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.Linear):
                module.weight.data.normal_(0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm1d):
                module.weight.data.normal_(1.0, 0.02)
                module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.normal_(1.0, 0.02)
                module.bias.data.zero_()

    def forward(self, img, labels):
        y_2 = labels.view(-1, self.num_class, 1, 1)
        y_2 = y_2.expand(-1, -1, img_w, img_h)

        x = torch.cat([img, y_2], 1)

        x = self.conv(x)
        x = x.view(-1, 128 * int((img_w / 4)) * int((img_h / 4)))
        x = self.fc(x)
        return x

In [50]:
def train_func(D_model, G_model, batch_size, z_dim, num_class, criterion, 
               D_optimizer, G_optimizer, data_loader, device):
    #訓練モード
    D_model.train()
    G_model.train()

    #本物のラベルは1
    y_real = torch.ones((batch_size, 1)).to(device)
    D_y_real = (torch.rand((batch_size, 1))/2 + 0.7).to(device) #Dに入れるノイズラベル

    #偽物のラベルは0
    y_fake = torch.zeros((batch_size, 1)).to(device)
    D_y_fake = (torch.rand((batch_size, 1)) * 0.3).to(device) #Dに入れるノイズラベル

    #lossの初期化
    D_running_loss = 0
    G_running_loss = 0

    #バッチごとの計算
    for batch_idx, (data, labels) in enumerate(data_loader):
        #バッチサイズに満たない場合は無視
        if data.size()[0] != batch_size:
            break

        #ノイズ作成
        z = torch.normal(mean = 0.5, std = 0.2, size = (batch_size, z_dim)) #平均0.5の正規分布に従った乱数を生成

        real_img, label, z = data.to(device), labels.to(device), z.to(device)

        #Discriminatorの更新
        D_optimizer.zero_grad()

        #Discriminatorに本物画像を入れて順伝播⇒Loss計算
        D_real = D_model(real_img, label)
        D_real_loss = criterion(D_real, D_y_real)

        #DiscriminatorにGeneratorにノイズを入れて作った画像を入れて順伝播⇒Loss計算
        fake_img = G_model(z, label)
        D_fake = D_model(fake_img.detach(), label) #fake_imagesで計算したLossをGeneratorに逆伝播させないように止める
        D_fake_loss = criterion(D_fake, D_y_fake)

        #2つのLossの和を最小化
        D_loss = D_real_loss + D_fake_loss

        D_loss.backward()
        D_optimizer.step()

        D_running_loss += D_loss.item()

        #Generatorの更新
        G_optimizer.zero_grad()

        #Generatorにノイズを入れて作った画像をDiscriminatorに入れて順伝播⇒見破られた分がLossになる
        fake_img_2 = G_model(z, label)
        D_fake_2 = D_model(fake_img_2, label)

        #Gのloss(max(log D)で最適化)
        G_loss = -criterion(D_fake_2, y_fake)

        G_loss.backward()
        G_optimizer.step()
        G_running_loss += G_loss.item()

    D_running_loss /= len(data_loader)
    G_running_loss /= len(data_loader)

    return D_running_loss, G_running_loss

In [51]:

import os
from IPython.display import Image
from torchvision.utils import save_image
%matplotlib inline

def Generate_img(epoch, G_model, device, z_dim, noise, var_mode, labels, log_dir='./logs_cGAN'):
    G_model.eval()

    with torch.no_grad():
        if var_mode == True:
            #生成に必要な乱数
            noise = torch.normal(mean = 0.5, std = 0.2, size = (num_label, z_dim)).to(device)
        else:
            noise = noise

        #Generatorでサンプル生成
        import pdb;pdb.set_trace()
        samples = G_model(noise, labels).data.cpu()
        samples = (samples/2)+0.5
        try:
          print(os.path.join(log_dir, 'epoch_%05d.png'))
          print(os.path.exists(log_dir))
          save_image(samples,os.path.join(log_dir, 'epoch_%05d.png' % (epoch)), nrow = 7)
        except:
          import pdb;pdb.set_trace()
        img = Image('./logs_cGAN/epoch_%05d.png' % (epoch))
        display(img)

In [52]:

#再現性確保のためseed値固定
SEED = 1111
random.seed(SEED)
np.random.seed(SEED) 
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def model_run(num_epochs, batch_size = batch_size, dataloader = train_loader, device = device):

    #Generatorに入れるノイズの次元
    z_dim = 30
    var_mode = False #表示結果を見るときに毎回異なる乱数を使うかどうか
    #生成に必要な乱数
    noise = torch.normal(mean = 0.5, std = 0.2, size = (num_label, z_dim)).to(device)

    #クラス数
    num_class = num_label

    #Generatorを試すときに使うラベルを作る
    labels = []
    for i in range(num_class):
        tmp = np.identity(num_class)[i]
        tmp = np.array(tmp, dtype = np.float32)
        labels.append(tmp)
    label = torch.Tensor(labels).to(device)

    #モデル定義
    D_model = Discriminator(num_class).to(device)
    G_model = Generator(z_dim, num_class).to(device)

    #lossの定義(引数はtrain_funcの中で指定)
    criterion = nn.BCELoss().to(device)

    #optimizerの定義
    D_optimizer = torch.optim.Adam(D_model.parameters(), lr=0.0002, betas=(0.5, 0.999), eps=1e-08, weight_decay=1e-5, amsgrad=False)
    G_optimizer = torch.optim.Adam(G_model.parameters(), lr=0.0002, betas=(0.5, 0.999), eps=1e-08, weight_decay=1e-5, amsgrad=False)

    D_loss_list = []
    G_loss_list = []

    all_time = time.time()
    for epoch in range(num_epochs):
        start_time = time.time()

        D_loss, G_loss = train_func(D_model, G_model, batch_size, z_dim, num_class, criterion, 
                                    D_optimizer, G_optimizer, dataloader, device)

        D_loss_list.append(D_loss)
        G_loss_list.append(G_loss)

        secs = int(time.time() - start_time)
        mins = secs / 60
        secs = secs % 60

        #エポックごとに結果を表示
        print('Epoch: %d' %(epoch + 1), " | 所要時間 %d 分 %d 秒" %(mins, secs))
        print(f'\tLoss: {D_loss:.4f}(Discriminator)')
        print(f'\tLoss: {G_loss:.4f}(Generator)')

        if (epoch + 1) % 1 == 0:
            Generate_img(epoch, G_model, device, z_dim, noise, var_mode, label)

        #モデル保存のためのcheckpointファイルを作成
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch':epoch,
                'model_state_dict':G_model.state_dict(),
                'optimizer_state_dict':G_optimizer.state_dict(),
                'loss':G_loss,
            }, './checkpoint_cGAN/G_model_{}'.format(epoch + 1))

    return D_loss_list, G_loss_list

#モデルを回す
D_loss_list, G_loss_list = model_run(num_epochs = 100)

Epoch: 1  | 所要時間 5 分 57 秒
	Loss: 0.9382(Discriminator)
	Loss: -0.4288(Generator)
> <ipython-input-51-21b8f46055ea>(18)Generate_img()
     16         #Generatorでサンプル生成
     17         import pdb;pdb.set_trace()
---> 18         samples = G_model(noise, labels).data.cpu()
     19         samples = (samples/2)+0.5
     20         try:

ipdb> noise
tensor([[0.2787, 0.5323, 0.3630,  ..., 0.3289, 0.6153, 0.5144],
        [0.6162, 0.6210, 0.3869,  ..., 0.8749, 0.9489, 1.0593],
        [0.2456, 0.4697, 0.6470,  ..., 0.7699, 0.4950, 0.4187],
        ...,
        [0.7263, 0.2793, 0.5867,  ..., 0.4621, 0.3470, 0.4157],
        [0.5121, 0.8131, 0.6434,  ..., 0.4014, 0.5223, 0.1114],
        [0.6198, 0.4195, 0.5730,  ..., 0.5135, 0.4866, 0.0438]],
       device='cuda:0')
ipdb> noise.shape
torch.Size([85, 30])
ipdb> labels.shape
torch.Size([85, 85])
ipdb> labels
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
       

BdbQuit: 

In [ ]:
int((img_h / 4))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(10,7))

loss = fig.add_subplot(1,1,1)

loss.plot(range(len(D_loss_list)),D_loss_list,label='Discriminator_loss')
loss.plot(range(len(G_loss_list)),G_loss_list,label='Generator_loss')

loss.set_xlabel('epoch')
loss.set_ylabel('loss')

loss.legend()
loss.grid()

fig.show()

In [ ]:
import cloudpickle
%matplotlib inline
#取り出すepochを指定する
point = 15

#モデルの構造を定義
z_dim = 30
num_class = num_label
G = Generator(z_dim = z_dim, num_class = num_class)

#checkpointを取り出す
checkpoint = torch.load('./checkpoint_cGAN/G_model_{}'.format(point))

#Generatorにパラメータを入れる
G.load_state_dict(checkpoint['model_state_dict'])

#検証モードにしておく
G.eval()

#pickleで保存
with open ('nenchan_cGAN.pkl','wb')as f:
    cloudpickle.dump(G,f)

In [53]:
# letter = 'あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわゐゑをんゝ'

# strs = input()
with open('nenchan_cGAN.pkl','rb')as f:
    Generator = cloudpickle.load(f)

# import pdb;pdb.set_trace()
noise = torch.normal(mean = 0.5, std = 0.2, size = (1, 30))
tmp = np.identity(num_label)
tmp = np.array(tmp, dtype = np.float32)
label = [tmp]

img = Generator(noise, torch.Tensor(label))
img = img.reshape((img_w,img_h))
# import pdb;pdb.set_trace()
img = img.detach().numpy()

cv2.imwrite('./sentence.png', img)
# img = Image('./sentence.png')
display(img)

RuntimeError: running_mean should contain 85 elements not 1500